In [7]:
import os
import shutil
import pandas as pd
from collections import defaultdict

In [8]:
# 이미지 파일명에서 성별 및 스타일을 추출하는 함수 (뒤의 W/M으로 성별 구분)
def extract_info_from_filename(filename):
    # 파일명 예시: "W_00237_60_popart_W.jpg"
    parts = filename.split('_')
    if len(parts) < 4:
        return None, None  # 형식이 맞지 않는 파일명은 무시
    style = parts[3]  # 스타일 정보는 네 번째 요소
    gender = '여성' if parts[-1].startswith('W') else '남성'  # 파일명의 마지막 부분이 성별을 나타냄
    return gender, style

In [9]:
# 디렉토리 내 파일명으로 통계 정보를 추출하는 함수
def generate_statistics(directory):
    # 성별 & 스타일별 이미지 수를 저장할 딕셔너리
    stats = defaultdict(lambda: defaultdict(int))
    
    # 디렉토리 내 모든 파일명에 대해 성별과 스타일 정보 추출
    for filename in os.listdir(directory):
        if filename.endswith(".jpg"):
            gender, style = extract_info_from_filename(filename)
            if gender and style:
                stats[gender][style] += 1
                
    # 통계 정보를 DataFrame으로 변환
    stats_list = []
    for gender, style_dict in stats.items():
        for style, count in style_dict.items():
            stats_list.append([gender, style, count])

    stats_df = pd.DataFrame(stats_list, columns=['성별', '스타일', '이미지 수'])
    return stats_df

In [10]:
# Training 및 Validation 데이터 경로 (경로는 실제 데이터셋 위치로 변경해야 합니다)
training_image_dir = '/home/gyuha_lee/DCC2024/dataset/training_image'
validation_image_dir = '/home/gyuha_lee/DCC2024/dataset/validation_image'

In [11]:
# Training 데이터 통계
training_stats_df = generate_statistics(training_image_dir)

# Validation 데이터 통계
validation_stats_df = generate_statistics(validation_image_dir)

In [12]:
# Training 데이터 통계표 출력
print("Training 데이터 통계")
print(f"{'성별':<4} {'스타일':<15} {'이미지 수':>10}")
print("-" * 34)
for index, row in training_stats_df.iterrows():
    print(f"{row['성별']:<4} {row['스타일']:<15} {row['이미지 수']:>10}")

# Validation 데이터 통계표 출력
print("\nValidation 데이터 통계")
print(f"{'성별':<4} {'스타일':<15} {'이미지 수':>10}")
print("-" * 34)
for index, row in validation_stats_df.iterrows():
    print(f"{row['성별']:<4} {row['스타일']:<15} {row['이미지 수']:>10}")

Training 데이터 통계
성별   스타일                  이미지 수
----------------------------------
남성   metrosexual            278
남성   ivy                    237
남성   sportivecasual         298
남성   mods                   269
남성   bold                   268
남성   hiphop                 274
남성   normcore               364
남성   hippie                 260
여성   kitsch                  91
여성   lounge                  45
여성   disco                   37
여성   bodyconscious           95
여성   sportivecasual         157
여성   hippie                  91
여성   normcore               153
여성   athleisure              67
여성   hiphop                  48
여성   lingerie                55
여성   oriental                78
여성   minimal                139
여성   feminine               154
여성   cityglam                67
여성   classic                 77
여성   punk                    65
여성   genderless              77
여성   powersuit              120
여성   grunge                  31
여성   ecology                 64
여성   space           

In [8]:
# 각각의 통계 정보를 CSV 파일로 저장
training_stats_df.to_csv("training_image_statistics.csv", index=False, encoding='utf-8-sig')
validation_stats_df.to_csv("validation_image_statistics.csv", index=False, encoding='utf-8-sig')

In [10]:
# 새 폴더 구조를 생성할 경로
organized_training_dir = '/home/gyuha_lee/DCC2024/dataset/organized_training_image'
organized_validation_dir = '/home/gyuha_lee/DCC2024/dataset/organized_validation_image'

In [11]:
# 폴더 정리 함수 정의 (성별 및 스타일별 폴더 생성 후 이미지 복사)
def organize_images_by_class_and_style(image_dir, target_dir):
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    # 각 이미지 파일명에서 성별 및 스타일 정보를 추출하고, 해당 폴더로 이미지를 복사
    for filename in os.listdir(image_dir):
        if filename.endswith(".jpg"):
            # 파일명에서 성별 및 스타일 정보 추출
            gender, style = extract_info_from_filename(filename)
            if gender and style:
                # 성별 폴더 경로 생성
                gender_folder = os.path.join(target_dir, gender)

                # 스타일 폴더 경로 생성
                style_folder = os.path.join(gender_folder, style)

                # 성별 및 스타일 폴더가 없으면 생성
                if not os.path.exists(gender_folder):
                    os.makedirs(gender_folder)
                if not os.path.exists(style_folder):
                    os.makedirs(style_folder)

                # 이미지 파일을 스타일 폴더로 복사
                source_path = os.path.join(image_dir, filename)
                target_path = os.path.join(style_folder, filename)
                shutil.copy(source_path, target_path)  # 이동 대신 복사

In [12]:
# 이미지 파일명에서 성별 및 스타일을 추출하는 함수
def extract_info_from_filename(filename):
    # 파일명 예시: "W_00237_60_popart_W.jpg"
    parts = filename.split('_')
    if len(parts) < 4:
        return None, None  # 형식이 맞지 않는 파일명은 무시
    gender = '여성' if parts[0] == 'W' else '남성'
    style = parts[3]
    return gender, style

In [ ]:
# Training 데이터 폴더 정리 (복사)
organize_images_by_class_and_style(training_image_dir, organized_training_dir)

# Validation 데이터 폴더 정리 (복사)
organize_images_by_class_and_style(validation_image_dir, organized_validation_dir)

print("이미지 폴더가 성별 및 스타일별로 정리되어 복사되었습니다.")

In [10]:
import torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim

In [ ]:
# GPU 사용 가능 여부 확인
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"사용 중인 장치: {device}")

In [ ]:
# 파일명에서 성별 및 스타일 정보를 추출하는 함수 (뒤의 W/M으로 성별 구분)
def extract_info_from_filename(filename):
    parts = filename.split('_')
    style = parts[3]  # 스타일 정보는 네 번째 요소
    gender = '여성' if parts[-1].startswith('W') else '남성'  # 마지막 부분에서 성별 구분
    return gender, style

In [ ]:
# Custom Dataset 클래스 정의
class CustomFashionDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(image_dir) if f.endswith(".jpg")]
        self.classes = self.get_classes()  # 클래스(성별 + 스타일) 정보를 정의
        
    def get_classes(self):
        # 고유한 (성별, 스타일) 조합을 찾기 위한 클래스 정의
        classes = set()
        for filename in self.image_files:
            gender, style = extract_info_from_filename(filename)
            classes.add((gender, style))
        return sorted(list(classes))  # 클래스 목록을 정렬된 리스트로 반환
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.image_dir, img_name)
        
        # 이미지 로드
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        
        # 성별 & 스타일 정보 추출 및 레이블 인코딩
        gender, style = extract_info_from_filename(img_name)
        label = self.classes.index((gender, style))  # 성별 & 스타일을 레이블로 변환
        
        return image, label


In [ ]:
# 데이터 전처리 설정
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
# Custom Dataset 사용
image_dir = './dataset/training_image'  # 폴더를 나누지 않은 경우
dataset = CustomFashionDataset(image_dir, transform=transform)

In [ ]:
# DataLoader 설정
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)

In [ ]:
# ResNet-18 모델 정의
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=False)
num_classes = len(dataset.classes)  # 클래스 수에 맞게 출력층 크기 조정
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

In [ ]:
# 손실 함수 및 최적화 함수 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# 모델 학습 및 검증 함수
def train_model(model, dataloader, criterion, optimizer, num_epochs=25):
    model.train()  # 학습 모드
    for epoch in range(num_epochs):
        running_loss = 0.0
        running_corrects = 0
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # 옵티마이저 초기화
            optimizer.zero_grad()
            
            # Forward
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            
            # Backward + 최적화
            loss.backward()
            optimizer.step()
            
            # 손실 값 및 정확도 계산
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        
        # 에포크 당 평균 손실 및 정확도 출력
        epoch_loss = running_loss / len(dataloader.dataset)
        epoch_acc = running_corrects.double() / len(dataloader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

In [ ]:
# 모델 학습
train_model(model, dataloader, criterion, optimizer, num_epochs=25)

In [ ]:
# 학습된 모델 저장
torch.save(model.state_dict(), 'resnet18_custom_dataset.pth')